In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 19: An Elephant Named Joseph ---</h2><p>The Elves contact you over a highly secure emergency channel. Back at the North Pole, the Elves are busy <span title="Eggnoggedly misunderstanding them, actually.">misunderstanding</span> <a href="https://en.wikipedia.org/wiki/White_elephant_gift_exchange">White Elephant parties</a>.</p>
<p>Each Elf brings a present. They all sit in a circle, numbered starting with position <code>1</code>. Then, starting with the first Elf, they take turns stealing all the presents from the Elf to their left.  An Elf with no presents is removed from the circle and does not take turns.</p>
<p>For example, with five Elves (numbered <code>1</code> to <code>5</code>):</p>
<pre><code>  1
5   2
 4 3
</code></pre>
<ul>
<li>Elf <code>1</code> takes Elf <code>2</code>'s present.</li>
<li>Elf <code>2</code> has no presents and is skipped.</li>
<li>Elf <code>3</code> takes Elf <code>4</code>'s present.</li>
<li>Elf <code>4</code> has no presents and is also skipped.</li>
<li>Elf <code>5</code> takes Elf <code>1</code>'s two presents.</li>
<li>Neither Elf <code>1</code> nor Elf <code>2</code> have any presents, so both are skipped.</li>
<li>Elf <code>3</code> takes Elf <code>5</code>'s three presents.</li>
</ul>
<p>So, with <em>five</em> Elves, the Elf that sits starting in position <code>3</code> gets all the presents.</p>
<p>With the number of Elves given in your puzzle input, <em>which Elf gets all the presents?</em></p>
</article>


In [2]:
from itertools import cycle


def bassie_elf(number_of_elfs: int) -> int:
    elfs = [1] * number_of_elfs

    for turn in cycle(range(number_of_elfs)):
        if elfs[turn]:
            j = 1
            while elfs[(turn + j) % number_of_elfs] == 0:
                j += 1
            elfs[turn] += elfs[(turn + j) % number_of_elfs]
            elfs[(turn + j) % number_of_elfs] = 0

            if elfs[turn] == number_of_elfs:
                return turn + 1
    return 0


tests_bassie_elf = [
    {
        "name": "Example 1",
        "number_of_elfs": 2,
        "expected": 1,
    },
    {
        "name": "Example 2",
        "number_of_elfs": 3,
        "expected": 3,
    },
    {
        "name": "Example 3",
        "number_of_elfs": 4,
        "expected": 1,
    },
    {
        "name": "Example 4",
        "number_of_elfs": 5,
        "expected": 3,
    },
]

run_tests_params(bassie_elf, tests_bassie_elf)


Test Example 1 passed, for bassie_elf.
Test Example 2 passed, for bassie_elf.
Test Example 3 passed, for bassie_elf.
Test Example 4 passed, for bassie_elf.
Success


                         1
                        /  \
                        \  /
                         2

-1- 1 takes presents from 2
Answer 1

                         1
                       /   \
                      3-----2

-1- 1 takes presents from 2
-2- 2 has no presents and will be skipped
-3- 3 will take all presents from 1
Answer 3

                     1----2
                     |    |
                     4----3

-1- 1 takes presents from 2
-2- 2 has no presents and will be skipped
-3- 3 will take all presents from 4
-4- 4 has no presents and will be skipped
-1- 1 will takes presents of 3
Answer 1


In [3]:
bassie_elf(3014603)

1834903

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1834903</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Realizing the folly of their present-exchange rules, the Elves agree to instead steal presents from the Elf <em>directly across the circle</em>. If two Elves are across the circle, the one on the left (from the perspective of the stealer) is stolen from.  The other rules remain unchanged: Elves with no presents are removed from the circle entirely, and the other elves move in slightly to keep the circle evenly spaced.</p>
<p>For example, with five Elves (again numbered <code>1</code> to <code>5</code>):</p>
<ul>
<li>The Elves sit in a circle; Elf <code>1</code> goes first:
<pre><code>  <em>1</em>
5   2
 4 3
</code></pre></li>
<li>Elves <code>3</code> and <code>4</code> are across the circle; Elf <code>3</code>'s present is stolen, being the one to the left. Elf <code>3</code> leaves the circle, and the rest of the Elves move in:
<pre><code>  <em>1</em>           1
5   2  --&gt;  5   2
 4 -          4
</code></pre></li>
<li>Elf <code>2</code> steals from the Elf directly across the circle, Elf <code>5</code>:
<pre><code>  1         1 
-   <em>2</em>  --&gt;     2
  4         4 
</code></pre></li>
<li>Next is Elf <code>4</code> who, choosing between Elves <code>1</code> and <code>2</code>, steals from Elf <code>1</code>:
<pre><code> -          2  
    2  --&gt;
 <em>4</em>          4
</code></pre></li>
<li>Finally, Elf <code>2</code> steals from Elf <code>4</code>:
<pre><code> <em>2</em>
    --&gt;  2  
 -
</code></pre></li>
</ul>
<p>So, with <em>five</em> Elves, the Elf that sits starting in position <code>2</code> gets all the presents.</p>
<p>With the number of Elves given in your puzzle input, <em>which Elf now gets all the presents?</em></p>
</article>

</main>


In [4]:
def bassie_elf_II_too_slow(n: int) -> int:
    elfs = list(range(1, n + 1))
    presents = [1] * n

    elf = 0
    while len(elfs) > 1:
        other_elf = (elf + len(elfs) // 2) % len(elfs)
        presents[elf] += presents[other_elf]
        elfs.pop(other_elf)
        presents.pop(other_elf)
        if other_elf < elf:
            elf -= 1
        elf = (elf + 1) % len(elfs)

    return elfs[0]


bassie_elf_II_too_slow(5)

2

In [5]:
def highest_power_of_3(n):
    option = 0
    while 3**option <= n:
        option += 1
    return 3 ** (option - 1)


def bassie_elf_II(n):
    # https://oeis.org/A334473
    x = highest_power_of_3(n)
    if x == n:
        return x
    if n < 2 * x:
        return n % x
    return x + 2 * (n % x)


assert all(bassie_elf_II(i) == bassie_elf_II_too_slow(i) for i in range(1, 101))

In [6]:
bassie_elf_II(3014603)

1420280

<link href="style.css" rel="stylesheet"></link>

<p>Your puzzle answer was <code>1420280</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
